# Codes for constructing the Plots/Figures
## Three Month Span

The goal is to have this notebook contain all codes that construct the plots  found in the paper.

## Initialize

In [1]:
run_list = [  # 'msis2'       ,
#                'tiegcm_oc'   ,
#                'dtm2020'     ,
#                'jb2008'      ,
               'hasdm_oc',
#                'ctipe_oc',
           ]


###  Input/Read GEODYN Runs and dump to pickle, or load the pickle

#### Three month

In [2]:
%load_ext autoreload
%autoreload 2

import sys  
import os
import pickle 
import gc

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn



for i,val in enumerate(run_list): 
    for ii in ['_p1', '_p2']:
        
        dir_save = '/data/zach_work/output_from_runs/icesat2_3monthrun_pickles/'
        pickle_file = dir_save+'Fulltime_valid_'+val+ii+'.pkl'

        if not os.path.exists(pickle_file):

            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            run_settings = '/data/zach_work/validation/ICESat2_3month_Validation/runsettings_3month_bwdrag_'+ val +ii+'.yaml'\
            ### Load the data into an object
            Obj_Geodyn = Pygeodyn(run_settings)
            Obj_Geodyn.getData_BigData_lowmemory()

            gc.collect()

            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(Obj_Geodyn, filehandler)
            filehandler.close()
            Obj_Geodyn = 0
            print('   ', 'Saved pickle')


        else:
            print(val,ii,'  Pickle created -- will load in next step', sep='')
            

### Load the data if the pickles exist
print()
print()
gc.collect()

Obj_3months = {}
for i,val in enumerate(run_list): #,'jb2008','msis2'
    for ii in ['_p1', '_p2']:
        
        
        dir_save = '/data/zach_work/output_from_runs/icesat2_3monthrun_pickles/'
        pickle_file = dir_save+'Fulltime_valid_'+val+ii+'.pkl'
        filehandler = open(pickle_file, 'rb') 
        Obj_3months[val+ii] = pickle.load(filehandler)
        filehandler.close()
        print('Loaded data... ', val+ii)


hasdm_oc_p1  Pickle created -- will load in next step
hasdm_oc_p2  Pickle created -- will load in next step


Loaded data...  hasdm_oc_p1
Loaded data...  hasdm_oc_p2


### Plot Settings and Features

In [3]:
plots_dir = '/data/zach_work/validation/ICESat2_3month_Validation/final_plots/'

In [4]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio   ### Allows you to save plotly figs


import pandas as pd
import datetime
import numpy as np

config = dict({
                'displayModeBar': False,
                'responsive': False,
                'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                })



import numpy as np
import pandas as pd
import os

# px.colors.colorscale_to_colors()
# plotly.colors.PLOTLY_SCALES["Viridis"]

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor
def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )




cols = get_color("Viridis", np.linspace(0, 1, 5))
map_cols = np.linspace(0, 1, 5)
colorscale=[]
for i,val in enumerate(map_cols):
    colorscale.append([val, cols[i]])
    



# Simplify Plotting Schemes:
col1 =  px.colors.qualitative.Plotly[2]
col2 =  px.colors.qualitative.Plotly[4]
col3 =  px.colors.qualitative.Plotly[1]
col4 =  px.colors.qualitative.Plotly[3]
col5 =  px.colors.qualitative.Plotly[4]
col6 =  px.colors.qualitative.Plotly[5]



# Simplify Plotting Schemes:
col_msis2 =  px.colors.qualitative.Plotly[2]
col_jb2008 =  px.colors.qualitative.Plotly[4]
col_dtm2020 =  px.colors.qualitative.Plotly[1]
col_tiegcm_oc =  px.colors.qualitative.Plotly[5]
col_hasdm_oc =  px.colors.qualitative.Plotly[0]
col_ctipe_oc =  px.colors.qualitative.Plotly[7]

x_annot_val = 1.1



def get_plot_params(plot_num, model_name_string):
    '''
    INPUT:   
        Plot number, model_name string, x_annot_val
    
    RETURN:
        col, x_annot, y_annot1, y_annot2, m_size,   
    '''
    
    
    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
        y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = x_annot_val
        y_annot1 = .8
        y_annot2 = .8
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = x_annot_val
        y_annot1 = .5
        y_annot2 = .7
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = x_annot_val
        y_annot1 = .2
        y_annot2 = .55
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .45
        col = col5
        m_size = 3.5
        
    elif plot_num == 5:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .35
        col = col6
        m_size = 3.5
        
    if model_name_string == 'msis2':
        col=col_msis2
    elif model_name_string == 'dtm2020':
        col=col_dtm2020
    elif model_name_string == 'jb2008':
        col=col_jb2008
    elif model_name_string == 'tiegcm_oc':
        col=col_tiegcm_oc
    elif model_name_string == 'hasdm_oc':
        col=col_hasdm_oc
    elif model_name_string == 'ctipe_oc':
        col=col_ctipe_oc
        
    ### Old Models
    elif model_name_string == 'dtm87':
        col='grey'
    elif model_name_string == 'jaachia71':
        col='grey'
    elif model_name_string == 'msis86':
        col='grey'
    elif model_name_string == 'msis00':
        col='tan'
        
    return(col,x_annot,y_annot1,y_annot2,m_size)
        

In [5]:
def show_named_plotly_colours():
    """
    function to display to user the colours to match plotly's named
    css colours.

    Reference:
        #https://community.plotly.com/t/plotly-colours-list/11730/3

    Returns:
        plotly dataframe with cell colour to match named colour name

    """
    s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, black, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
    li=s.split(',')
    li=[l.replace('\n','') for l in li]
    li=[l.replace(' ','') for l in li]

    import pandas as pd
    import plotly.graph_objects as go

    df=pd.DataFrame.from_dict({'colour': li})
    fig = go.Figure(data=[go.Table(
      header=dict(
        values=["Plotly Named CSS colours"],
        line_color='black', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[df.colour],
        line_color=[df.colour], fill_color=[df.colour],
        align='center', font=dict(color='black', size=11)
      ))
    ])

    fig.show()

# show_named_plotly_colours() 

### Some plotting functions

In [6]:



def orb_avg(den_df, arc):
    
    
    #### Find the index for the correct date
    vals  = np.arange(den_df[arc].index[0],den_df[arc].index[-1]+1)
    df = den_df[arc].set_index('Date',drop=False ) 
    df['i_vals'] = vals
    index_date = df.loc[df.index.max()]['i_vals'].min()
    
#     print('index_date', index_date)
    lat = np.asarray(den_df[arc]['Lat'][:index_date])
    time_pd = pd.to_datetime(den_df[arc]['Date'][:index_date])
    i = np.nonzero( lat[1:]*lat[0:-1]  <  np.logical_and(0 , lat[1:] > lat[0:-1] )  )
    i = i[0]

    d_avg = np.zeros(np.size(i))
    height_avg = np.zeros(np.size(i))
    
#     print('time_pd',time_pd)

    time_avg = []
    d_avg_rolling = []
    
    roll_avg_count = 0
    for j in range(np.size(i)-1):
        d_avg[j]      = np.mean(den_df[arc]['rho (kg/m**3)'  ][i[j] : i[j+1]-1  ]  )
        height_avg[j] = np.mean(den_df[arc]['Height (meters)'][i[j] : i[j+1]-1  ]  )
#         mean_time      = np.mean(time_pd[   i[j] : i[j+1]-1  ])
        t1 = pd.to_datetime(time_pd[ i[j]    ])
        t2 = pd.to_datetime(time_pd[ i[j+1]-1])
        datemiddle = pd.Timestamp(t1) + (pd.Timestamp(t2) - pd.Timestamp(t1)) / 2

        time_avg.append(datemiddle)

        if roll_avg_count ==1:
            d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
            roll_avg_count =0
            
        roll_avg_count+=1 
    d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
        
    return(time_avg, d_avg, d_avg_rolling )
    

def plot_density_orbit_avg(fig, obj_m1, plot_num ):
    

    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        fig.add_trace(go.Scattergl(x=time_avg,
                                 y=d_avg_rolling,
#                                  y=d_avg,
                                name= model_m1 ,
                                mode='markers',
                                marker=dict(
                                color=col,
                                size=7,),
                                showlegend=False,
                                   ),
                                   row=1, col=1,
                                   )
        

        fig.update_yaxes(type="log", exponentformat= 'power',row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="kg/m^3", row=1, col=1)
#     fig.update_yaxes(title_text="nT", row=2, col=1)
#     fig.update_yaxes(title_text="sfu", row=3, col=1)
    fig.update_layout(legend= {'itemsizing': 'constant'})
#     fig.update_layout(
#         font=dict(          size=18, ),
#         autosize=False,
#         width=900,
#         height=1000,)
    
    return(fig)



def Plot_Densitycomparison(fig, obj_m1, plot_num):


    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input'][:]):
        
        
        #### INDEX THE DENSITY DF correctly
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        str_run_param = 'run_parameters'+ arc
        final_iter = obj_m1.__dict__[str_run_param]['str_iteration']
        i_arc = ii+1
        
        
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()       
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        
        print('----',model_m1,'----')
        print('     mean:    ',np.mean(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print('     variance:',np.std(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print()
#         if ii ==0:
#             fig.add_trace(go.Scattergl(  x=obj_m1.Density[arc]['Date'][:index_date][:],
#                                      y=obj_m1.Density[arc]['rho (kg/m**3)'][:index_date][:],
#                                      name= model_m1,
#                                      mode='markers',
#                                      opacity=1,
#                                      marker=dict(
#                                         color=col, 
#                                         size=m_size,),
#                                      showlegend=True,),
#                                       secondary_y=False,
#                                        row=1, col=1,)

#         else:
#             fig.add_trace(go.Scattergl(  x=obj_m1.Density[arc]['Date'][:index_date][:],
#                          y=obj_m1.Density[arc]['rho (kg/m**3)'][:index_date][:],
#                          name= model_m1,
#                          mode='markers',
#                          opacity=1,
#                          marker=dict(
#                             color=col, 
#                             size=m_size,),
#                          showlegend=False,),
#                           secondary_y=False,
#                            row=1, col=1,)
        fig.add_trace(go.Scattergl(x=time_avg,
                     y=d_avg_rolling,
                    name= model_m1 ,
                    mode='markers',
                    marker=dict(
                    color='black',
                    size=5,),
                    showlegend=False,),
                       row=1, col=1,)


#         (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')

#         fig = add_stats_annotation(fig, model_m1+'<br>Mean='+ str(np.round(mean,4))+'<br>RMS='+ str(np.round(rms_about_zero,4)), col , x_annot, y_annot1)


    fig.update_yaxes( title=r"$\frac{kg}{m^3}$", type='log', exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Date", row=1, col=1)

    return(fig)


def legend_as_annotation(fig, den_model_string, color_it, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="paper",
            yref="paper",
            showarrow=False,
            text=den_model_string,
            font=dict(
                size=16,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=color_it,
            opacity=0.9
            )

    return(fig)

def add_stats_annotation(fig, text_in, col, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="x domain",
            yref="y domain",
            showarrow=False,
            text=text_in,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=col,
            opacity=0.9
            )
    return(fig)

import pandas as pd

def STATS_residuals(residuals,measurement_type):
    import numpy as np
    n = np.size(residuals)
    mean = (1/n)*(np.sum(residuals))
    variance = (1/n)*(np.sum(np.square(residuals)))
    rms = np.sqrt(variance)
    rms_about_zero = np.sqrt((n/(n-1))*variance)
    
                
#     print('mean            ',measurement_type ,':',mean)
#     print('rms             ',measurement_type ,':',rms)
#     print('rms about zero  ',measurement_type ,':',rms_about_zero)
#     print()
    
    return(mean,rms,rms_about_zero)






### Load GPI data

In [7]:
from netCDF4 import Dataset
def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


arc_list = []

arc_list_18 = np.arange(292,366)
for i in arc_list_18:
    val = '2018'+str(i)
    arc_list.append(int(val))
    
    #     print(val)
    
arc_list_19 = np.arange(1,10)
for i in arc_list_19:
    val = '201900'+str(i)
    arc_list.append(int(val))


path_to_f107 = '/data/data_geodyn/gpi_1960001-2021243_f107aDaily.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)

date = []
kp_list = []
f107d_list = []
f107a_list  = []
date_3hr = []


for i,val in enumerate(arc_list):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    
    date.append(pd.to_datetime( str(val), format='%Y%j'))

    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        
solar_fluxes = {}
solar_fluxes['f107d_list'] = f107d_list
solar_fluxes['f107a_list'] = f107a_list
solar_fluxes['date']       = date
solar_fluxes['date_3hr']   = date_3hr
solar_fluxes['kp_expand']  = kp_expand


### Prepare RMS total Plot arrays

arc_listlist=[  ['2018.292', '2018.293', '2018.294', '2018.295', '2018.296', 
                 '2018.297', '2018.298', '2018.299' ],                  
                #
                ['2018.304', '2018.305', '2018.306', '2018.307', '2018.308' ],  
                #
                ['2018.313', '2018.314', '2018.315', '2018.316', '2018.317',
                 '2018.318', '2018.319', '2018.320', '2018.321', '2018.322',
                 '2018.323', '2018.324', '2018.325', '2018.326', '2018.327' ],  
                #
                ['2018.335', '2018.336', '2018.337' ],  
                #
                ['2018.349', '2018.350', '2018.351', '2018.352' ],  
                #
                ['2018.356', '2018.357', '2018.358' ],  
                #
                ['2018.365', '2019.001', '2019.002', '2019.003', '2019.004', 
                 '2019.005', '2019.006', '2019.007', '2019.008',
                '2019.009'],  
                ]


# Plot Full Time Period


## Solar Flux and Kp

In [8]:
    
    
    
# fig = make_subplots(
#     rows=2, cols=1,
# #     subplot_titles=([ 'Kp','F10.7']),
#     shared_xaxes=True,
#     vertical_spacing = 0.1,
#     )

# fig.add_trace(go.Scatter(  x=date,
#                              y=f107d_list,
#                              name= 'F107d',
#                              mode='markers+lines',
#                              opacity=1,
#                              marker=dict( color='grey', size=2, ),
#                              line = dict(shape = 'hv', color = 'grey', width=2),
#                              showlegend=False,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )

# fig.add_trace(go.Scatter(  x=date,
#                              y=f107a_list,
#                              name= 'F107a',
#                              mode='markers+lines',
# #                              opacity=1,
#                              marker=dict(color='black',size=2),
#                              line = dict(shape = 'hv', color = 'black', width=2),
#                              showlegend=False,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )

# fig.add_trace(go.Scatter(  x=date_3hr,
#                              y=kp_expand,
#                              name= 'Kp',
#                              mode='markers+lines',
# #                              opacity=1,
#                              marker=dict(color='cornflowerblue',size=2,),
#                              line = dict(shape = 'hv', color = 'cornflowerblue', width=2),
#                              showlegend=False,
#                           ),
#                               secondary_y=False,
#                                row=2, col=1,
#                           )

#     #### Arc Background 
# fig.add_vrect(x0=pd.to_datetime( str(2018292), format='%Y%j'),
#               x1=pd.to_datetime( str(2018299), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018304), format='%Y%j'),
#               x1=pd.to_datetime( str(2018308), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018313), format='%Y%j'),
#               x1=pd.to_datetime( str(2018327), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018335), format='%Y%j'),
#               x1=pd.to_datetime( str(2018337), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018349), format='%Y%j'),
#               x1=pd.to_datetime( str(2018352), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018356), format='%Y%j'),
#               x1=pd.to_datetime( str(2018358), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)
# fig.add_vrect(x0=pd.to_datetime( str(2018365), format='%Y%j'),
#               x1=pd.to_datetime( str(2019008), format='%Y%j'),
#               fillcolor='LightSkyBlue',
#               opacity=.5,
#               layer="below",
#               line_width=0)




# # fig.update_layout(
# #         #     title= 'NGDC Geophysical Indices --  Kp and F10.7',
# #                 autosize=False, ,
# #                 font=dict(size=14),
# #                 legend= {'itemsizing': 'constant'})

# fig.update_yaxes( title=r"$\text{F}_{\text{10.7}}\text{ Solar Flux (sfu)}$", row=1, col=1)
# fig.update_yaxes( title=r"$\text{K}_\text{p}\text{ Index}$", row=2, col=1)
# fig.update_xaxes( title=r"$\text{Date}$", row=2, col=1)

# font_dict=dict(family='Arial',size=16,color='black')
# fig.update_xaxes(showline=True,
#                  showticklabels=False,
#                  linecolor='black',
#                  linewidth=1,
#                  ticks='inside',
#                  tickfont=font_dict,
#                  mirror='allticks',
#                  tickwidth=1.4,
#                  tickcolor='black',
#                  gridcolor='lightgray',
#                  row=1, col=1
#                 )

# fig.update_xaxes(showline=True,
#                  showticklabels=True,
#                  linecolor='black',
#                  linewidth=1,
#                  ticks='inside',
#                  tickfont=font_dict,
#                  mirror='allticks',
#                  tickwidth=1.4,
#                  tickcolor='black',
#                  gridcolor='lightgray',
#                  row=2, col=1)


# # fig.update_yaxes(range=[1.15, 1.35], row=1,col=1)
# # fig.update_yaxes(range=[1.15, 1.35], row=1,col=2)
# fig.update_yaxes(showline=True,      # add line at x=0
#                  linecolor='black',  # line color
#                  linewidth=1,      # line size
#                  ticks='inside',    # ticks outside axis
#                  tickfont=font_dict, # tick label font
#                  mirror='allticks',  # add ticks to top/right axes
#                  tickwidth=1.4,      # tick width
#                  tickcolor='black',  # tick color
#                   gridcolor='lightgray')

# fig.update_annotations(font_size=18)  # Increase size of subplot title
# fig.update_layout(
# #                   title = '',
#                   autosize=False,    width=700,    height=500,
# #                   legend= {'itemsizing': 'constant'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.show(config=config)
# pio.write_image(fig, plots_dir+'geophysical_indices.pdf')



## Model Validation Metrics

In [9]:
# solar_fluxes_shaded = {}
# solar_fluxes_shaded['f107d_list'] = f107d_list
# solar_fluxes_shaded['f107a_list'] = f107a_list
# solar_fluxes_shaded['date']       = date
# solar_fluxes_shaded['date_3hr']   = date_3hr
# solar_fluxes_shaded['kp_expand']  = kp_expand


### Prepare RMS total Plot arrays
arc_list={}
arc_list['1']=  ['2018.292', '2018.293', '2018.294', '2018.295', '2018.296', 
                 '2018.297', '2018.298', '2018.299' ]                  
                #
arc_list['2']=  ['2018.304', '2018.305', '2018.306', '2018.307', '2018.308' ]  
                #
arc_list['3']=  ['2018.313', '2018.314', '2018.315', '2018.316', '2018.317',
                 '2018.318', '2018.319', '2018.320', '2018.321', '2018.322',
                 '2018.323', '2018.324', '2018.325', '2018.326', '2018.327' ]  
                #
arc_list['4']=  ['2018.335', '2018.336', '2018.337' ]  
                #
arc_list['5']=  ['2018.349', '2018.350', '2018.351', '2018.352' ]  
                #
arc_list['6']=  ['2018.356', '2018.357', '2018.358' ]  
                #
arc_list['7']=  ['2018.365', '2019.001', '2019.002', '2019.003', '2019.004', 
                '2019.005', '2019.006', '2019.007', '2019.008',
                '2019.009' ]  



# for arclist in arc_listlist:
#     for arc in arclist:
#         print(pd.to_datetime( str(arc), format='%Y.%j'))
shadezone_i = {} 
shadezone_f = {}
shadezone_i['1'] = '2018.299'
shadezone_f['1'] = '2018.304'
shadezone_i['2'] = '2018.308'
shadezone_f['2'] = '2018.313'
shadezone_i['3'] = '2018.327'
shadezone_f['3'] = '2018.335'
shadezone_i['4'] = '2018.337'
shadezone_f['4'] = '2018.349'
shadezone_i['5'] = '2018.352'
shadezone_f['5'] = '2018.356'
shadezone_i['6'] = '2018.358'
shadezone_f['6'] = '2018.365'

SF = {}
for iii in np.arange(1,8):
#     print(iii)
    SF[str(iii)+'_shade_f107d'] = []
    SF[str(iii)+'_shade_date']  = []
    SF[str(iii)+'_f107d']       = []
    SF[str(iii)+'_date']        = []
    #     
    SF[str(iii)+'_shade_date_3hr'] = []
    SF[str(iii)+'_shade_kp_expand']  = []
    SF[str(iii)+'_date_3hr']       = []
    SF[str(iii)+'_kp_expand']        = []
    # 
#     shadezone_i[str(iii)] = 
#     shadezone_f[str(iii)] = 
    
    
    
for i, val in enumerate(solar_fluxes['date']):
    for iii in np.arange(1,8):

        if iii == 7:
            pass
        else:
            if val >= pd.to_datetime(shadezone_i[str(iii)], format='%Y.%j') and val <= pd.to_datetime(shadezone_f[str(iii)], format='%Y.%j'):    
                SF[str(iii)+'_shade_f107d'].append(solar_fluxes['f107d_list'][i])
                SF[str(iii)+'_shade_date'].append(val)

        if val >= pd.to_datetime(arc_list[str(iii)][0], format='%Y.%j') and val <= pd.to_datetime(arc_list[str(iii)][-1], format='%Y.%j'):    
            SF[str(iii)+'_f107d'].append(solar_fluxes['f107d_list'][i])
            SF[str(iii)+'_date'].append(val)

for i, val in enumerate(solar_fluxes['date_3hr']):
    for iii in np.arange(1,8):

        if iii == 7:
            pass
        else:
            if val >= pd.to_datetime(shadezone_i[str(iii)], format='%Y.%j') and val <= pd.to_datetime(shadezone_f[str(iii)], format='%Y.%j'):    
                SF[str(iii)+'_shade_date_3hr'].append(val)
                SF[str(iii)+'_shade_kp_expand'].append(solar_fluxes['kp_expand'][i])

        if val >= pd.to_datetime(arc_list[str(iii)][0], format='%Y.%j') and val <= pd.to_datetime(arc_list[str(iii)][-1], format='%Y.%j'):    
#             print(val)
            SF[str(iii)+'_date_3hr'].append(val)
            SF[str(iii)+'_kp_expand'].append(solar_fluxes['kp_expand'][i])



In [10]:
# SF[str(iii)+'_date_3hr']


In [11]:
# fig = make_subplots(rows=1, cols=1,
# #                     subplot_titles=([ ' ', ]),
#                     #                     
#                     specs=[[ {"secondary_y": True} ]],
#                     #
#                     vertical_spacing = 0.05,
#                     shared_xaxes=True)
# for iii in np.arange(1,8):
#     fig.add_trace(go.Scatter(x=SF[str(iii)+'_shade_date'],
#                                y=SF[str(iii)+'_shade_f107d'],
#                                mode='lines',
#                                opacity=0.5,
# #                                        marker=dict(color='cornflowerblue',size=3),
#                                    line = dict(dash='dot',shape = 'hv', color = 'cornflowerblue', width=2),
#                                showlegend=False),
#                                secondary_y=True,row=1, col=1)
#     fig.add_trace(go.Scatter(x=SF[str(iii)+'_date'],
#                                y=SF[str(iii)+'_f107d'],
#                                mode='lines',
#                                opacity=1,
#                                    line = dict(shape = 'hv', color = 'cornflowerblue', width=2),
#                                showlegend=False),
#                                secondary_y=True,row=1, col=1)
#     fig.add_trace(go.Scatter(x=SF[str(iii)+'_shade_date_3hr'],
#                                y=SF[str(iii)+'_shade_kp_expand'],
#                                mode='lines',
#                                opacity=0.5,
# #                                        marker=dict(color='cornflowerblue',size=3),
#                                    line = dict(dash='dot',shape = 'hv', color = 'grey', width=1),
#                                showlegend=False),
#                                secondary_y=False,row=1, col=1)
#     fig.add_trace(go.Scatter(x=SF[str(iii)+'_date_3hr'],
#                                y=SF[str(iii)+'_kp_expand'],
#                                mode='lines',
#                                opacity=1,
#                                    line = dict(shape = 'hv', color = 'black', width=1),
#                                showlegend=False),
#                                    secondary_y=False,row=1, col=1)
# fig.show(config=dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }))


In [ ]:
# %%capture

%load_ext autoreload
%autoreload 2

index1 = 0
index2 = -1
index3h_1 = 0
index3h_2 = -1

def plot_all_full(fig, obj_m1, plot_num, SF,    arc_listlist ):
    

    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
        
    #### -----------------------------------------------------------------------------------------------------
    if plot_num == 2 or plot_num == 0:
        #### Solar Flux Plot
        for iii in np.arange(1,8):
            fig.add_trace(go.Scattergl(x=SF[str(iii)+'_shade_date'],
                                       y=SF[str(iii)+'_shade_f107d'],
                                       mode='lines',
                                       opacity=0.5,
    #                                        marker=dict(color='cornflowerblue',size=3),
                                           line = dict(dash='dot',shape = 'hvh', color = 'cornflowerblue', width=2),
                                       showlegend=False),
                                       secondary_y=True,row=1, col=1)
            fig.add_trace(go.Scattergl(x=SF[str(iii)+'_date'],
                                       y=SF[str(iii)+'_f107d'],
                                       mode='lines',
                                       opacity=1,
                                           line = dict(shape = 'hvh', color = 'cornflowerblue', width=2),
                                       showlegend=False),
                                       secondary_y=True,row=1, col=1)
            fig.add_trace(go.Scattergl(x=SF[str(iii)+'_shade_date_3hr'],
                                       y=SF[str(iii)+'_shade_kp_expand'],
                                       mode='lines',
                                       opacity=0.5,
    #                                        marker=dict(color='cornflowerblue',size=3),
                                           line = dict(dash='dot',shape = 'hvh', color = 'grey', width=1),
                                       showlegend=False),
                                       secondary_y=False,row=1, col=1)
            fig.add_trace(go.Scattergl(x=SF[str(iii)+'_date_3hr'],
                                       y=SF[str(iii)+'_kp_expand'],
                                       mode='lines',
                                       opacity=1,
                                           line = dict(shape = 'hvh', color = 'black', width=1),
                                       showlegend=False),
                                       secondary_y=False,row=1, col=1)
        #### Background data gaps
        fig.add_vrect(x0=pd.to_datetime( str(2018290), format='%Y%j'),
                      x1=pd.to_datetime( str(2018292), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018300), format='%Y%j'),
                      x1=pd.to_datetime( str(2018304), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018309), format='%Y%j'),
                      x1=pd.to_datetime( str(2018313), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018329), format='%Y%j'),
                      x1=pd.to_datetime( str(2018335), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018338), format='%Y%j'),
                      x1=pd.to_datetime( str(2018349), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018353), format='%Y%j'),
                      x1=pd.to_datetime( str(2018356), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2018359), format='%Y%j'),
                      x1=pd.to_datetime( str(2018365), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)
        fig.add_vrect(x0=pd.to_datetime( str(2019009), format='%Y%j'),
                      x1=pd.to_datetime( str(2019010), format='%Y%j'),
                      fillcolor='gainsboro',
                      opacity=.6,
                      layer="below",
                      line_width=0,
                     row=1,col=1)

    
    for i1,arc1 in enumerate(arc_listlist):#obj_m1.__dict__['global_params']['arc_input']):
        dateplot = []
        rms_totals = []
        
        for ii,arc in enumerate(arc1):#obj_m1.__dict__['global_params']['arc_input']):
            if arc not in obj_m1.__dict__['global_params']['arc_input']:
                continue
            else:
                arc =arc+'.01'

                dateplot.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
                rms_totals.append(obj_m1.__dict__['Statistics'][arc]['T_RMS'].values[0])
                ### -----------------------------------------------------------------------------------------------------
                ###     DENSITY
                ###
                ## Remove the denisty file duplication
                vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
                df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
                df['i_vals'] = vals
                index_date = df.loc[df.index.max()]['i_vals'].min()
                time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
                #
                
                ### -----------------------------------------------------------------------------------------------------
                ###     Orbit Averaged Density
                fig.add_trace(go.Scattergl(x=time_avg,
                                           y=d_avg_rolling,
                                           ### name= model_m1,
                                           mode='markers+lines',
                                           opacity=1,
                                               marker=dict(color=col,size=2),
                                           ###     line = dict( color = col, width=2),
                                               line = dict(shape='hvh', dash ='solid', color = col, width=2),
                                           showlegend=False), row=2, col=1)
                ### -----------------------------------------------------------------------------------------------------
                ###     In Track Residuals
                data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']
                fig.add_trace(go.Scattergl(x=data_resids['Date'][::75],
                                           y=data_resids['T'][::75],
                                           ###   name= model_m1,
                                             mode='markers+lines',
                                             opacity=1,
                                                 marker=dict(color=col,size=2),
                                                 line = dict( color = col, width=2),
                                             showlegend=False),
                                             secondary_y=False, row=3, col=1)
            ### -----------------------------------------------------------------------------------------------------
            ###     TOTAL RMS
            fig.add_trace(go.Scattergl(x=dateplot,
                                       y=rms_totals,
                                           ###e= 'NTW '+model_m1,
                                       mode='markers+lines',
                                             opacity=1,
                                             marker=dict(color=col,size=6),
                                             line = dict(shape='hvh', dash ='solid', color = col, width=2),
                                       showlegend=False),row=4, col=1)
    
    if plot_num == 2 or plot_num == 0:
        ###
        ### DENSITY AXIS
        fig.update_yaxes(title_text=r"$\text{Orbit Averaged Density } (\frac{kg}{m^3})$", 
                         type="log", exponentformat= 'power',row=2, col=1)
        ###
        ### InTrack Residual Axis
        fig.update_yaxes( title=r"$\text{In-Track Residuals (m)}$",
    #                      range=[-100,55],
                         exponentformat= 'power',row=3, col=1)
        ###
        ### RMS AXIS
        fig.update_yaxes( title=r"$\text{In-Track RMSe (m)}$" ,type="linear" , exponentformat= 'power', row=4, col=1)
        ###
        ###  DATE on Final x-Axis only
        fig.update_xaxes(title=r"$\text{Date}$", 
                         range=[pd.to_datetime( "181018-000000", format='%y%m%d-%H%M%S'),
                                pd.to_datetime( "190109-120000", format='%y%m%d-%H%M%S')],
                         row=4, col=1)

    return(fig)

    
    
    
    
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------
    ### -----------------------------------------------------------------------------------------------------




fig = make_subplots(rows=4, cols=1,
#                     subplot_titles=([ ' ', ]),
                    #                     
                    specs=[[ {"secondary_y": True} ],
                           [ {"secondary_y": False} ],
                           [ {"secondary_y": False} ],
                           [ {"secondary_y": False} ]],
                    #
                    vertical_spacing = 0.05,
                    shared_xaxes=True)



# fig = plot_all_full(fig, Obj_3months['msis2_p1'],        0, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['msis2_p2'],        0, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['tiegcm_oc_p1'],    3, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['tiegcm_oc_p2'],    3, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['dtm2020_p1'],   1, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['dtm2020_p2'],   1, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['jb2008_p1'],    2, SF, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['jb2008_p2'],    2, SF, arc_listlist)
fig = plot_all_full(fig, Obj_3months['hasdm_oc_p1'],  2, SF, arc_listlist)
fig = plot_all_full(fig, Obj_3months['hasdm_oc_p2'],  2, SF, arc_listlist)



# fig = plot_all_full(fig, Obj_3months['hasdm_oc'],  4, solar_fluxes, arc_listlist)
# fig = plot_all_full(fig, Obj_3months['ctipe_oc'],  5, solar_fluxes, arc_listlist)




#######################################################
font_dict=dict(family='Arial',size=14,color='black')
#######################################################

fig.update_xaxes(showline=True,
             showticklabels=True,
             linecolor='black',
             linewidth=1,
             ### Major ticks
                 ticks='inside',
                 tickfont=font_dict,
                 mirror=True,
                 tickwidth=2,
                 ticklen=9,
                 tickcolor='grey',
                 tick0="2018-10-20" ,
                 dtick=86400000.0*7,    # milliseconds in a day, every 7 days
                 #### Minor Ticks
                 minor=dict(dtick=86400000.0, # milliseconds in a day
                       tickwidth=1,
                       ticklen=4,
                       tickcolor='grey',
                       ticks='inside'),             
                 gridcolor='gainsboro',
                 showgrid=False,
                 layer='above traces',
                 tickangle=0,
                 row=1, col=1)
fig.update_yaxes(title_text=r"$\text{F}_{\text{10.7}}\text{ Solar Flux (sfu)}$", 
                 color='cornflowerblue',
                 range=[64, 86],
                 showline=True,      # add line at x=0
                 linecolor='cornflowerblue',  # line color
                 linewidth=1,        # line size
                 ticks='inside',     # ticks outside axis
                 tickfont=dict(family='sans-serif',size=14,color='cornflowerblue'), # tick label font
                 mirror=False,  # add ticks to top/right axes
                 tickwidth=1,      # tick width
                 tickcolor='cornflowerblue',   # tick color
                 gridcolor='gainsboro',
                 showgrid=False,
                 anchor="x", overlaying="y", side="right",
                 layer='above traces',
                 secondary_y=True, row=1, col=1,)
fig.update_yaxes(title_text=r"$\text{K}_\text{p}\text{ Index}$",
                 secondary_y=False,
                 showline=True,      # add line at x=0
                 linecolor='black',  # line color
                 linewidth=1,        # line size
                 ticks='inside',     # ticks outside axis
                 tickfont=dict(family='sans-serif',size=14,color='black'), # tick label font
                 mirror=False,  # add ticks to top/right axes
                 tickwidth=1,      # tick width
                 tickcolor='black',   # tick color
                 gridcolor='gainsboro',
                 showgrid=False,
                 layer='above traces',
                 row=1, col=1,)
    

for i in [2,3,4]:
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=True,
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
                      tickwidth=2,
                      ticklen=9,
                      tickcolor='grey',
                      tick0="2018-10-20" ,
                      dtick=86400000.0*7,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=0.5,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=0.5,
                     layer='above traces',
                     row=i, col=1)

    


#### LEGEND ####
modelnames = [  "MSISe2",  "DTM2020",  "JB2008",  "TIEGCM",  "HASDM",  "CTIPe"]
df = pd.DataFrame({"starts_colors": [col_msis2, col_dtm2020, col_jb2008, 
                                     col_tiegcm_oc, col_hasdm_oc, col_ctipe_oc]})
fig.update_traces(showlegend=False).add_traces(
    [   go.Scattergl(name=modelnames[i], 
               x=[pd.to_datetime( "181107-000000", format='%y%m%d-%H%M%S')],
               mode='lines',
               line = dict(shape = 'hvh',  width=25),
               marker_color=c, 
               showlegend=True)
        for i,c in enumerate((df.loc[:,["starts_colors"]].values.ravel()))])
fig.update_layout(legend=dict(
    yanchor="middle",
    y=0.63,
    xanchor="center",
    x=1.015,
        font=font_dict,
        bgcolor="white",
        bordercolor="darkgrey",
        borderwidth=0.8,))

fig.update_layout(
#                   title = '',
                  autosize=False,    width=1000,    height=1100,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=14 )  # Increase size of subplot title

fig.show(renderer="jpg",
            config=dict({
                'displayModeBar': False,
                'responsive': False,
                'staticPlot': True,
                'displaylogo': False,
                'showTips': False,
                }))

# pio.write_image(fig, plots_dir+'threemonth_fullresult.jpg', scale=3)

# x=0
# fig=0
# print(x)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-450aa9a5b051>", line 371, in <module>
    fig.show(renderer="jpg",
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/basedatatypes.py", line 3398, in show
    return pio.show(self, *args, **kwargs)
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_renderers.py", line 388, in show
    bundle = renderers._build_mime_bundle(fig_dict, renderers_string=renderer, **kwargs)
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_renderers.py", line 296, in _build_mime_bundle
    bundle.update(renderer.to_mimebundle(fig_dict))
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_base_renderers.py", line 127, in to_mimebundle
    image_bytes = to_image(
  File "

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-450aa9a5b051>", line 371, in <module>
    fig.show(renderer="jpg",
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/basedatatypes.py", line 3398, in show
    return pio.show(self, *args, **kwargs)
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_renderers.py", line 388, in show
    bundle = renderers._build_mime_bundle(fig_dict, renderers_string=renderer, **kwargs)
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_renderers.py", line 296, in _build_mime_bundle
    bundle.update(renderer.to_mimebundle(fig_dict))
  File "/data/miniconda3/envs/pygeodyn/lib/python3.8/site-packages/plotly/io/_base_renderers.py", line 127, in to_mimebundle
    image_bytes = to_image(
  File "